In [1]:
from tx import TxTest
import unittest

suite = unittest.TestLoader().loadTestsFromModule(TxTest())
unittest.TextTestRunner().run(suite)


...........sss
----------------------------------------------------------------------
Ran 14 tests in 0.014s

OK (skipped=3)


<unittest.runner.TextTestResult run=14 errors=0 failures=0>

In [2]:
from helper import HelperTest
import unittest

suite = unittest.TestLoader().loadTestsFromModule(HelperTest())
unittest.TextTestRunner().run(suite)

....ss
----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK (skipped=2)


<unittest.runner.TextTestResult run=6 errors=0 failures=0>